In [36]:
import os, re
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import treebank
from nltk.corpus import dependency_treebank
from nltk.grammar import DependencyGrammar
from nltk.parse import DependencyGraph
import pprint
from nltk.parse.corenlp import CoreNLPDependencyParser
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import numpy as np
import re
from numpy.linalg import norm

In [37]:
ps = PorterStemmer()
wnl = WordNetLemmatizer()
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')

In [38]:
class Habitat:
    oid = -1
    name = ""

In [39]:
train_path = "./BioNLP-OST-2019_BB-norm_train/"
test_path = "./BioNLP-OST-2019_BB-norm_test/"
dev_path = "./BioNLP-OST-2019_BB-norm_dev/"
BB_train = os.listdir(train_path)
BB_test = os.listdir(test_path)
BB_dev = os.listdir(dev_path)
obp_file = "OntoBiotope_BioNLP-OST-2019.obo"

In [40]:
ontology = open(obp_file,encoding="utf8").read()

In [41]:
BB_train_a1 = sorted([name for name in BB_train if name.endswith("a1")])
BB_train_a2 = sorted([name for name in BB_train if name.endswith("a2")])
BB_train_txt = sorted([name for name in BB_train if name.endswith("txt")])

BB_dev_a1 = sorted([name for name in BB_dev if name.endswith("a1")])
BB_dev_a2 = sorted([name for name in BB_dev if name.endswith("a2")])
BB_dev_txt = sorted([name for name in BB_dev if name.endswith("txt")])

BB_test_a1 = sorted([name for name in BB_test if name.endswith("a1")])
BB_test_txt = sorted([name for name in BB_test if name.endswith("txt")])

In [42]:
def original_phrase(phrase):
    return phrase

In [43]:
def stem_phrase(phrase):
    #print(type(phrase))
    #print(phrase)
    
    phrase = phrase.split(" ")
    for i in range(len(phrase)):
        phrase[i] = ps.stem(phrase[i])
    phrase = " ".join(phrase)
    return phrase

In [44]:
def lemmatize_phrase(phrase):
    #print(type(phrase))
    #print(phrase)
    
    phrase = phrase.split(" ")
    for i in range(len(phrase)):
        phrase[i] = wnl.lemmatize(phrase[i])
    phrase = " ".join(phrase)
    return phrase

In [45]:
phrase_reducer = lemmatize_phrase

In [46]:
def get_headword(phrase):
    #print(phrase)
    if phrase == phrase.upper():
        return ""
    
    parses = dep_parser.parse(phrase.split())
    triples = [[(governor, dep, dependent) for governor, dep, dependent in parse.triples()] for parse in parses]
    
    try:
        if(len(triples) > 0 and len(triples[0]) > 0 and len(triples[0][0]) > 0 and len(triples[0][0][0]) > 0 and triples[0][0][0][0] is not None):
            return triples[0][0][0][0]
        else:
            return phrase
    except:
        return phrase
    
    

In [47]:
'asdsa-asdas asda-asda'.replace('-', ' ')

'asdsa asdas asda asda'

In [48]:
def add_documents(txt, processor):
    document = []
    txt = processor(txt)
    sentence_list = sent_tokenize(txt)
    
    for sentence in sentence_list:
        sentence = sentence.replace('-', ' ')
        word_list = word_tokenize(sentence)
        word_list = [word for word in word_list if word not in stopwords.words("english") and word.isalnum()]
        for i in range(len(word_list)):
            if word_list[i].isnumeric():
                word_list[i] = "##number##"
            if word_list[i] != word_list[i].upper():
                word_list[i] = processor(word_list[i].lower())
             
        document.append(word_list)
    #print(document)
    return document

In [49]:
documents = []
for i in range(len(BB_train_txt)):
    txt = open(train_path + BB_train_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)
for i in range(len(BB_dev_txt)):
    txt = open(dev_path + BB_dev_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)
for i in range(len(BB_test_txt)):
    txt = open(test_path + BB_test_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)

In [50]:
model = Word2Vec(documents, min_count=1)

In [51]:
phrase_reducer('Nares')

'Nares'

In [52]:
model['naris']

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 1.6954151e-03,  4.2707073e-03,  3.1701084e-03,  2.7947444e-03,
        4.0453440e-03,  4.0315003e-03, -2.4355885e-03,  3.8473727e-06,
        3.1842301e-03,  2.5977271e-03,  1.5356223e-03,  2.6943977e-03,
       -7.1773212e-04,  2.9759775e-03,  2.1087714e-03, -2.8029531e-03,
        9.8999368e-04, -2.8347650e-03,  2.4773302e-03,  1.9774116e-03,
       -4.7520613e-03,  7.2720001e-04, -1.6488311e-03,  3.2496378e-03,
       -4.2906539e-03, -3.6295014e-03,  6.5014430e-04,  1.2893652e-03,
       -2.3922857e-03,  2.9643141e-03, -3.8802396e-03,  1.7087054e-03,
       -1.5648431e-03,  1.0403885e-03,  1.5400667e-03,  9.2878554e-04,
       -4.9340571e-03, -1.8326401e-03, -2.1916411e-03,  2.0383529e-03,
        9.1619010e-04, -3.5289563e-03,  5.5387471e-04,  5.2349153e-03,
       -4.0744529e-03, -2.0950651e-04,  2.2264551e-03, -2.1171202e-03,
        4.9931239e-03, -1.8779733e-03,  2.4551477e-03, -1.0551532e-03,
        1.5099144e-03,  4.2983885e-03, -5.1042973e-03, -4.3728305e-05,
      

In [53]:
len(model.wv.vocab)

5537

In [54]:
type(list(model.wv.vocab))

list

In [55]:
len(model[list(model.wv.vocab)[0]])

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


100

In [56]:
def get_phrase_embedding(phrase, model, processor):
    original_phrase = phrase
    phrase = phrase.replace('-', ' ')
    hyphenless_phrase = phrase
    phrase = nltk.word_tokenize(phrase)
    
    phrase = [processor(word) for word in phrase if word.lower() not in stopwords.words("english") and word.isalnum()]
    for i in range(len(phrase)):
        if phrase[i].isnumeric():
            phrase[i] = "##number##"
    
    phrase_size = len(phrase)
    vec_size = len(model[list(model.wv.vocab)[0]])
    embedded_sum = np.zeros(vec_size)
    if(phrase_size == 0):
        return -1
    for word in phrase:
        if word not in list(model.wv.vocab):
            return -1
        embedded_sum = embedded_sum + np.array(model[word])
    embedded_sum = embedded_sum / phrase_size
    
    """hw = get_headword(hyphenless_phrase)
    if hw == "":
        print(hyphenless_phrase)
    else:
        hw_emb = np.array(model[processor(hw)])
        embedded_sum = (4*embedded_sum + hw_emb)/5"""
    
    return embedded_sum
    

In [57]:
def generate_habitat_map(processor, model):
    habitat_list = ontology.split("\n\n[Term]\n")[1:]
    habitat_map = {}
    habitat_map_originals = {}
    habitat_embeddings = {}
    for h in habitat_list:
        #new_habitat = Habitat()
        h = h.split("\n")
        oid = re.findall(r"(?<=id: OBT:)[0-9]+", h[0])[0]
        name = (re.findall(r"(?<=name: ).+", h[1])[0])
        original_name = name
        if name != name.upper():
            name = name.lower()
            name = processor(name)
    
        habitat_map[name] = oid
        habitat_map_originals[oid] = name
        embedding = get_phrase_embedding(name, model, processor)
        if type(embedding) == type(np.array([1])):
            habitat_embeddings[name] = embedding
    
        # word2vec buraya eklenmeli 
        for h_line in h:
            if "synonym" in h_line:
                synonym_name = (re.findall(r"(?<=synonym: \").+(?=\")", h_line)[0])
                if synonym_name != synonym_name.upper():
                    synonym_name = synonym_name.lower()
                    synonym_name = processor(synonym_name)
                if "EXACT" in h_line and synonym_name not in habitat_map:
                    habitat_map[synonym_name] = oid
                    if type(embedding) == type(np.array([1])):
                        habitat_embeddings[synonym_name] = embedding
                    
    return habitat_map, habitat_map_originals, habitat_embeddings

In [58]:
habitat_map, habitat_map_originals, habitat_embeddings = generate_habitat_map(phrase_reducer, model)

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [59]:
len(habitat_map)

4000

In [60]:
len(habitat_embeddings)

938

In [61]:
def add_habitats(a1, a2, processor, model):
    a2 = a2.split("\n")
    a1 = a1.split("\n")
    
    
    for annotation in a2:
        a1_line = 0
        t = ""
        oid = ""
        #print(annotation)
        if "OntoBiotope" in annotation:
            t = re.findall(r"(?<=Annotation:)T[0-9]+", annotation)[0]
            oid = re.findall(r"(?<=Referent:OBT:)[0-9]+", annotation)[0]
            named_entity = ""
            
            while t not in a1[a1_line] and a1_line < len(a1)-1:
                a1_line += 1
            if "Habitat" in a1[a1_line]: 
                named_entity = (a1[a1_line].split("\t")[2])
                named_entity = named_entity.replace('-', ' ')
                if named_entity != named_entity.upper():
                    named_entity = named_entity.lower()
                    named_entity = processor(named_entity)
                if named_entity not in habitat_map:
                    habitat_map[named_entity] = oid
                
                if habitat_map_originals[oid] not in habitat_embeddings:
                    habitat_embeddings[habitat_map_originals[oid]] = []
                
                if type(habitat_embeddings[habitat_map_originals[oid]]) == list:
                    emb = get_phrase_embedding(named_entity, model, processor)
                    if type(emb) == int:
                        print("lol")
                    habitat_embeddings[habitat_map_originals[oid]].append(emb)

In [62]:
for i in range(len(BB_train_txt)):    
    a1 = open(train_path + BB_train_a1[i],encoding="utf8").read()
    a2 = open(train_path + BB_train_a2[i],encoding="utf8").read()
    add_habitats(a1, a2, phrase_reducer, model)

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


lol
lol


In [63]:
for key in habitat_embeddings:
    if type(habitat_embeddings[key]) == list:
        vec_size = len(habitat_embeddings[key][0])
        temp_vec = np.zeros(vec_size)
        for vec in habitat_embeddings[key]:
            temp_vec += vec
        temp_vec /= len(habitat_embeddings[key])
        habitat_embeddings[key] = temp_vec

In [64]:
len(habitat_map_originals)

3602

In [65]:
habitat_embeddings

{'fermented cheese': array([ 0.1033549 , -0.06518824,  0.10813541,  0.04103443, -0.0348875 ,
        -0.00297241,  0.07038013,  0.00901779,  0.01096653,  0.05142654,
        -0.0118536 ,  0.04673142,  0.00698112,  0.02433076, -0.06097768,
        -0.04559442, -0.00359851,  0.01212336, -0.01000991,  0.12269961,
        -0.06901031, -0.00350222, -0.01196041, -0.0610927 , -0.04799344,
        -0.03793608, -0.02684952,  0.05757117,  0.03296405, -0.01986421,
         0.08491076, -0.06954343, -0.00646946, -0.04666196,  0.0605495 ,
        -0.13900126, -0.03466854, -0.03387603,  0.04874405,  0.05314941,
        -0.00946202,  0.01450153,  0.14834396,  0.09594524,  0.09156715,
         0.15077268,  0.03133052,  0.02615322,  0.04517541, -0.03484296,
         0.03236221, -0.01898239,  0.04492895,  0.04087842, -0.02738146,
         0.01876614,  0.10442598, -0.02841625,  0.0051179 , -0.09850057,
        -0.13787157, -0.08449515,  0.01451199,  0.04678898, -0.09917014,
        -0.07714391,  0.0251103

In [66]:
def predict_habitats_exact_matching(a1, txt, a1_name, processor, pred_file_name):
    pred_file = open(pred_file_name, "w")
    cands = 0
    matches = 0
    match_list = {}
    a1 = a1.split("\n")
    count = 1
    for line in a1:
        found = False
        if "Habitat" in line:
            cands += 1
            name = (line.split("\t")[2])
            named_entity = name
            if name != name.upper():
                name = name.lower()
                named_entity = processor(name)
            if named_entity in habitat_map:
                matches += 1
                match = name + " - " + habitat_map[named_entity] + "---exact"
                match_list[line.split("\t")[0]] = [habitat_map[named_entity], named_entity, "exact"]
                found = True
            else:
                #print(name)
                pre_named_entity = get_headword(name)
                #print(pre_named_entity)
                named_entity = processor(pre_named_entity)
                if named_entity in habitat_map:
                    matches += 1
                    match = name + " - " + habitat_map[named_entity] + "---headwordexact"
                    print(match)
                    match_list[line.split("\t")[0]] = [habitat_map[named_entity], name, "headwordexact", named_entity]
                    found = True
        if found:
            pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_list[line.split("\t")[0]][0] + "\n")
            count += 1
    #print(a1_name)
    #print("Out of", cands, "candidates,", matches, "matches found")
    #print()
    pred_file.close()
    return match_list

In [110]:
def predict_habitats(a1, txt, a1_name, processor, pred_file_name, model):
    pred_file = open(pred_file_name, "w")
    cands = 0
    matches = 0
    match_list = {}
    a1 = a1.split("\n")
    count = 1
    for line in a1:
        found = False
        if "Habitat" in line:
            name = (line.split("\t")[2])
            named_entity = name
            if name != name.upper():
                name = name.lower()
                named_entity = processor(name)
            if named_entity in habitat_map:
                matches += 1
                match = name + " - " + habitat_map[named_entity] + " - " + habitat_map_originals[habitat_map[named_entity]] + " ---exact"
                #print(match)
                match_list[line.split("\t")[0]] = [habitat_map[named_entity], named_entity, "exact"]
                pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_list[line.split("\t")[0]][0] + "\n")
                count += 1
                continue
            #print(name)
            pre_named_entity = get_headword(name)
            #print(pre_named_entity)
            named_entity = processor(pre_named_entity)
            if named_entity in habitat_map:
                matches += 1
                match = name + " - " + habitat_map[named_entity] + " - " + habitat_map_originals[habitat_map[named_entity]] +" ---headwordexact"
                #print(match)
                match_list[line.split("\t")[0]] = [habitat_map[named_entity], name, "headwordexact", named_entity]
                found = True
                pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_list[line.split("\t")[0]][0] + "\n")
                count += 1
                continue
            
            cands += 1
            name = (line.split("\t")[2])
            name = name.replace("-", " ")
            named_entity = name
            entity_vec = get_phrase_embedding(named_entity, model, processor)
            match_id = ""
            if type(entity_vec) != int:
                largest_prod = np.dot(entity_vec, habitat_embeddings[list(habitat_embeddings)[0]])/(norm(entity_vec)*norm(habitat_embeddings[list(habitat_embeddings)[0]]))
                match_id = habitat_map[list(habitat_embeddings)[0]]
                for habitat in habitat_embeddings:
                    prod = np.dot(entity_vec,habitat_embeddings[habitat])/(norm(entity_vec)*norm(habitat_embeddings[habitat]))
                    if prod > largest_prod:
                        largest_prod = prod
                        match_id = habitat_map[habitat]
            else:
                #print("no embedding for:", named_entity)
                continue
                
            match_list[line.split("\t")[0]] = [match_id, named_entity, "w2v"]
            match = name + " - " + match_id + " - " + habitat_map_originals[match_id] + " ---w2v"
            #print(line.split("\t")[0], "matched with w2v")
            #print(match_list)
            #print(match)
            pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_id + "\n")
            count += 1
            
    #print(a1_name)
    #print("Out of", cands, "candidates,", matches, "matches found")
    #print()
    pred_file.close()
    return match_list

In [111]:
annotations_test = {}
for i in range(len(BB_dev_txt)):
    a1 = open(dev_path + BB_dev_a1[i],encoding="utf8").read()
    txt = open(dev_path + BB_dev_txt[i],encoding="utf8").read()
    pred_file_name = "./dev_preds/" + BB_dev_a2[i]
    annotations_test[BB_dev_a1[i]] = predict_habitats(a1, txt, BB_dev_a1[i], phrase_reducer, pred_file_name, model)

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [112]:
annotations_test

{'BB-norm-10496597.a1': {'T3': ['001792', 'gastric', 'exact'],
  'T4': ['001577', 'gastric mucosa', 'exact'],
  'T5': ['000196',
   'gastric mucosal-associated lymphoid tissue',
   'headwordexact',
   'tissue'],
  'T6': ['001129', 'gastric mucosal', 'w2v'],
  'T7': ['001792', 'gastric', 'exact'],
  'T8': ['000972', 'MALT', 'w2v'],
  'T9': ['000061', 'monoclonal b cells', 'headwordexact', 'cell'],
  'T11': ['001792', 'gastric', 'exact'],
  'T12': ['003215', 'gastric malt', 'headwordexact', 'malt'],
  'T13': ['003220',
   'patients with atypical lymphoid infiltrates',
   'headwordexact',
   'patient'],
  'T14': ['003220',
   'patients with low-grade malt lymphoma',
   'headwordexact',
   'patient'],
  'T15': ['003220',
   'patients with helicobacter pylori-chronic active gastritis',
   'headwordexact',
   'patient'],
  'T16': ['003220',
   'patients with high-grade primary gastric lymphoma',
   'headwordexact',
   'patient'],
  'T17': ['001792', 'gastric', 'exact'],
  'T18': ['000972', '

In [113]:
def test_habitats(a1, a2, processor):
    
    a2 = a2.split("\n")
    a1 = a1.split("\n")
    match_list = {}
    for annotation in a2:
        a1_line = 0
        t = ""
        oid = ""
        #print(annotation)
        if "OntoBiotope" in annotation:
            t = re.findall(r"(?<=Annotation:)T[0-9]+", annotation)[0]
            oid = re.findall(r"(?<=Referent:OBT:)[0-9]+", annotation)[0]
            named_entity = ""
            while t not in a1[a1_line] and a1_line < len(a1)-1:
                a1_line += 1
            if "Habitat" in a1[a1_line]:
                named_entity = (a1[a1_line].split("\t")[2])
                if named_entity != named_entity.upper():
                    named_entity = named_entity.lower()
                    named_entity = processor(named_entity)
                match_list[t] = [oid, named_entity]
    return match_list

In [114]:
dev_a2 = {}
for i in range(len(BB_dev_txt)):
    a1 = open(dev_path + BB_dev_a1[i],encoding="utf8").read()
    a2 = open(dev_path + BB_dev_a2[i],encoding="utf8").read()
    dev_a2[BB_dev_a1[i]] = test_habitats(a1,a2, phrase_reducer)

In [115]:
annotations_test

{'BB-norm-10496597.a1': {'T3': ['001792', 'gastric', 'exact'],
  'T4': ['001577', 'gastric mucosa', 'exact'],
  'T5': ['000196',
   'gastric mucosal-associated lymphoid tissue',
   'headwordexact',
   'tissue'],
  'T6': ['001129', 'gastric mucosal', 'w2v'],
  'T7': ['001792', 'gastric', 'exact'],
  'T8': ['000972', 'MALT', 'w2v'],
  'T9': ['000061', 'monoclonal b cells', 'headwordexact', 'cell'],
  'T11': ['001792', 'gastric', 'exact'],
  'T12': ['003215', 'gastric malt', 'headwordexact', 'malt'],
  'T13': ['003220',
   'patients with atypical lymphoid infiltrates',
   'headwordexact',
   'patient'],
  'T14': ['003220',
   'patients with low-grade malt lymphoma',
   'headwordexact',
   'patient'],
  'T15': ['003220',
   'patients with helicobacter pylori-chronic active gastritis',
   'headwordexact',
   'patient'],
  'T16': ['003220',
   'patients with high-grade primary gastric lymphoma',
   'headwordexact',
   'patient'],
  'T17': ['001792', 'gastric', 'exact'],
  'T18': ['000972', '

In [116]:
true_pred_count = 0
false_pred_count = 0
total_count = 0
for key in dev_a2:
    """print()
    print(key)
    print(dev_a2[key])
    print("**************")"""
    total_count += len(dev_a2[key])
    if key in annotations_test:
        for pred in annotations_test[key]:            
            if annotations_test[key][pred][0] == dev_a2[key][pred][0]:
                true_pred_count += 1
                if annotations_test[key][pred][2] == "w2v":
                    print(key)
                    #print("Prediction:", annotations_test[key][pred]," Real:", dev_a2[key][pred], " Type:", annotations_test[key][pred][2])
                    print("Word:", annotations_test[key][pred][1]," Prediction:", habitat_map_originals[annotations_test[key][pred][0]], " Real:", habitat_map_originals[dev_a2[key][pred][0]], " Type:", annotations_test[key][pred][2])
                    print()
            else:
                if annotations_test[key][pred][2] == "w2v":
                    print(key)
                    #print("Prediction:", annotations_test[key][pred]," Real:", dev_a2[key][pred], " Type:", annotations_test[key][pred][2])
                    print("Word:", annotations_test[key][pred][1]," Prediction:", habitat_map_originals[annotations_test[key][pred][0]], " Real:", habitat_map_originals[dev_a2[key][pred][0]], " Type:", annotations_test[key][pred][2])
                    print()
                false_pred_count += 1

BB-norm-10496597.a1
Word: gastric mucosal  Prediction: gastric body  Real: gastric mucosa  Type: w2v

BB-norm-10496597.a1
Word: MALT  Prediction: asteraceae  Real: lymphatic system part  Type: w2v

BB-norm-10496597.a1
Word: MALT  Prediction: asteraceae  Real: lymphatic system part  Type: w2v

BB-norm-10496597.a1
Word: CD20  Prediction: contaminated sediment  Real: lymphocyte  Type: w2v

BB-norm-10496597.a1
Word: CD3  Prediction: stream sediment  Real: lymphocyte  Type: w2v

BB-norm-10496597.a1
Word: MALT  Prediction: asteraceae  Real: lymphatic system part  Type: w2v

BB-norm-10496597.a1
Word: lymphocytic  Prediction: urea solution  Real: lymphocyte  Type: w2v

BB-norm-10496597.a1
Word: lymphocytic  Prediction: urea solution  Real: lymphocyte  Type: w2v

BB-norm-10496597.a1
Word: gastric biopsies  Prediction: gastric mucosa  Real: medical sample  Type: w2v

BB-norm-10496597.a1
Word: CD20  Prediction: contaminated sediment  Real: lymphocyte  Type: w2v

BB-norm-10496597.a1
Word: CD3  Pre

In [95]:
recall = true_pred_count/total_count
recall

0.49508196721311476

In [96]:
precision = true_pred_count/(true_pred_count + false_pred_count)
precision

0.668141592920354

## Denemeler